In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
from kite_trade import *
import os
# import ta
from tqdm import tqdm
import util_fun as uf
from enctoken import get_kite
kite = get_kite()
import sqlite3
from swifter import set_defaults
set_defaults(
    progress_bar=False,

)
import warnings
warnings.filterwarnings("ignore")

In [3]:
timerframe_list = [
"day",
"minute",
 "3minute",
 "5minute",
 "10minute",
 "15minute",
 "30minute",
 "60minute",]

i=1
error_list = []
# instument in nse
inst = pd.DataFrame(kite.instruments("NSE"))
inst_filter = inst.query('(name != "")').copy()
inst_filter.rename(columns = {"tradingsymbol":'Symbol'},inplace = True)
# inst_filter.query("Symbol == 'HDFCBANK'")

nifty_500 = pd.read_csv('../data/ind_nifty500list.csv')
nifty_500 = nifty_500.merge(inst_filter, on = 'Symbol')
# inst_filter = inst_filter.query('segment == "NSE" and instrument_type == "EQ"')
inst_dict = dict(zip(nifty_500.Symbol, nifty_500.instrument_token))


In [4]:
def get_day_data(kite, instrument, time_frame, start_date,end_date):
    results = pd.DataFrame()
    df = pd.DataFrame(kite.historical_data(instrument,start_date, end_date,time_frame))
    results = results.append(df) 
    return results

def get_data_parllel(kite, instrument, time_frame, start_date, end_date):
    df_final = pd.DataFrame()
    temp_sd = pd.to_datetime(start_date).strftime('%Y-%m-01')
    if (pd.to_datetime(end_date) - pd.to_datetime(start_date)).days > 28:
        df = pd.DataFrame(pd.date_range(temp_sd ,end_date,freq="MS")).rename(columns={0:'start_date'})
        df['end_date'] = df.shift(-1)
        df['end_date'].fillna(pd.to_datetime(end_date), inplace=True)
    else:
        df = pd.DataFrame({"start_date": pd.to_datetime(start_date), "end_date": pd.to_datetime(end_date)}, index=[0])
    
    df['tf'] = time_frame
    df['inst'] = instrument
    df['fetch_data'] = df.swifter.apply(lambda x: pd.DataFrame(kite.historical_data(x['inst'], x['start_date'], x['end_date'], x['tf'])),axis = 1)
    for i in df['fetch_data']:
        df_final = df_final.append(i)
    df_final.rename(columns = {'date': "Date",'close':'Close', 'high':'High', 'low':"Low", 'open': "Open","volume":"Volume"}, inplace = True)
    df_final= df_final[(pd.to_datetime(df_final['Date'].dt.date) >= pd.to_datetime(start_date)) &  \
                      (pd.to_datetime(df_final['Date'].dt.date) <= pd.to_datetime(end_date))]
    df_final.set_index('Date', inplace=True)
    return df_final

## data pull

In [5]:
today_date = '2023-05-30'
start_dt = '2023-05-30'
end_dt = '2023-05-31'
time_frame = '15minute'

# import sql 
# conn = sql.create_connection('../data/Nifty_500.db')
# cursor_obj = conn.cursor()
# sql.delete_table(conn,'fivteen_minute')
df_all = pd.DataFrame()
for symbol, instument in tqdm(inst_dict.items()):
    if i:
        # print(symbol, instument)
        try:
            df_all = df_all.append(get_data_parllel(kite, instument, time_frame , start_dt,end_dt).assign(stock = symbol))
            # df_day['stock'] = symbol
            # df_all = df_all.append(df_day)
            # df_all.reset_index(drop = True)
            # df_day.to_sql('fivteen_minute',conn, if_exists="append",index = True, index_label = 'Date')
            
                # directory = f'../data/historical/{time_frame}/{symbol}'
                # if not os.path.exists(directory):
                #     os.makedirs(directory)
                # df_day.to_parquet(f'{directory}/part0.parquet')
        except:
                print(instument)
                error_list.append(instument)

    # break

    i += 1
print(error_list)


100%|██████████| 497/497 [02:38<00:00,  3.13it/s]

[]


In [6]:
df_all.to_parquet('data_30may_15min.parquet')
df_all = pd.read_parquet('data_30may_15min.parquet')

In [7]:
# df_all = sql.sql_to_pd(conn,"fivteen_minute" )
df_all = df_all.reset_index()
df_all['Date']  = pd.to_datetime(df_all['Date']).dt.tz_localize(None)
df_all['date_only'] = pd.to_datetime(df_all['Date'].dt.date)

In [8]:
df_all.head()

,Date,Open,High,Low,Close,Volume,stock,date_only
0,2023-05-30 09:00:00,24160.00,24160.00,24160.00,24160.00,0,3MINDIA,2023-05-30
1,2023-05-30 09:15:00,24160.00,24300.00,24160.00,24234.25,115,3MINDIA,2023-05-30
2,2023-05-30 09:30:00,24234.25,24244.55,24190.00,24239.00,41,3MINDIA,2023-05-30
3,2023-05-30 09:45:00,24239.00,24240.00,24171.15,24199.95,40,3MINDIA,2023-05-30
4,2023-05-30 10:00:00,24199.95,24220.00,24193.30,24220.00,57,3MINDIA,2023-05-30


In [ ]:
# df_all = df_all[df_all.stock.isin(nifty_500.Symbol.unique())]

## 6 candle breakdown

In [9]:
def first_six_candle_setup(df):
    try:
        # print(df.loc[df.Date.dt.time == pd.to_datetime('09:15:00').time(),'High'].unique()[0] )
        f_high = df.loc[df.Date.dt.time == pd.to_datetime('09:15:00').time(),'High'].unique()[0] 
        f_low = df.loc[df.Date.dt.time == pd.to_datetime('09:15:00').time(),'Low'].unique()[0]
        df['f_high'] = f_high
        df["f_low"] = f_low
        df['per_chg'] = (abs(f_high - f_low)/ f_low) * 100
        df['break_f_high'] = np.where(df['High'] > df['f_high'], "brk", 'not_b')
        df['break_f_low'] = np.where(df['Low'] < df['f_low'],"brk", 'not_b')
        df['break_any_side'] = np.where((df['break_f_high'] == 'not_b') & (df['break_f_low'] == 'not_b'),'not_b',"brk") 
        # print(df)
        # return df
        first_five_candle = df.loc[df.Date.dt.time <= pd.to_datetime('10:30:00').time(),'break_any_side'].unique()
        for i in first_five_candle:
            if 'brk' in i:
                return [False,df['per_chg'].unique()[0]]
        return [True,df['per_chg'].unique()[0]]
    except:
        return


In [10]:
# df_wipro = df_all[(df_all.date_only == '2023-05-05') & (df_all.stock == 'CGCL')]
# df_filter_w = df_wipro.groupby(['date_only','stock']).apply(first_six_candle_setup).reset_index().rename(columns={0:"flag"})

df_filter = df_all.groupby(['date_only','stock']).apply(first_six_candle_setup).reset_index().rename(columns={0:"flag"})
df_filter = pd.concat([df_filter.drop('flag', axis= 1),df_filter.flag.apply(pd.Series).rename(columns = {0 :'flag', 1: 'per_chg'})],axis = 1)

# sort by first candle minimum % chg
df_selected = df_filter[(df_filter.flag) & (df_filter.date_only == today_date)].sort_values('per_chg')#.head(50)
df_first_candle = df_all.loc[df_all.Date.dt.time == pd.to_datetime('09:15:00').time(),:]
df_selected = df_selected.merge(df_first_candle, how  = 'left', on = ['date_only','stock'])
df_selected['ltp_cols'] = "NSE" + ":" +df_selected['stock']
df_selected = df_selected[df_selected.Open.between(100,5000)]

In [11]:
import time
import datetime
# import xlwings as xw
# wb=xw.Book("../data/five_candle.xlsx")
# ws = wb.sheets["Sheet1"]

alrady_break_up = []
alrady_break_down = []
i = 0 
while(True):
    df_selected['ltp'] = df_selected['ltp_cols'].map(kite.ltp(df_selected.ltp_cols.unique().tolist())).apply(lambda x: x['last_price'])
    
    # if i == 0:
    #     print(i)
    #     i +=1
    #     df_selected_1 = df_selected[~((df_selected['ltp'] < df_selected['High']) &  (df_selected['ltp'] > df_selected['Low']))]
    #     df_selected_1['break_time'] = datetime.datetime.now().time()
    #     alrady_break.append(df_selected_1.stock.unique().tolist())
    df_selected['quantity'] =(45000/df_selected['ltp']) * 5
    df_selected['near_High'] = df_selected['High'] - df_selected['High']  * .001
    df_selected['near_low'] = df_selected['Low'] + df_selected['Low']  * .001
    df_selected_up = df_selected[(df_selected['ltp'] > df_selected['High'])]
    df_selected_down  =  df_selected[(df_selected['ltp'] < df_selected['Low'])]
    if i != 0:
        alert_list = list(set(df_selected_up.stock.unique().tolist()) - set(alrady_break_up))  + list(set(df_selected_down.stock.unique().tolist()) - set(alrady_break_down))
        if alert_list != []:
            print("sending message")
            # uf.alert_msg(df_selected[df_selected.stock.isin(alert_list)][['stock','ltp','quantity']],0,0)
            uf.telegram_msg(df_selected[df_selected.stock.isin(alert_list)][['stock','ltp','quantity']].round(0).values)

    alrady_break_up = df_selected_up.stock.unique().tolist()
    alrady_break_down = df_selected_down.stock.unique().tolist()
    
    # df_selected_2 = df_selected_2.merge(df_selected_1[['date_only','stock','break_time']], how = 'left', on = ['date_only','stock'])
    # df_selected_2['break_time'] = np.where(df_selected_2.stock.isin(alrady_break),df_selected_2['break_time'],datetime.datetime.now().time() )
    # alrady_break.append(df_selected_1.stock.unique().tolist())
    # df_selected_2['break_time'] = df_selected_2['break_time'].astype(str)
    print(df_selected_up.stock.unique().tolist())
    print(df_selected_down.stock.unique().tolist())
    i += 1
    # ws.clear_contents()
    # ws.range("A1").options(index=False).value = df_selected_up
    # ws.range(f"A{df_selected_up.shape[0]+3}").options(index=False).value = df_selected_down
    time.sleep(30)
    # break


['SUNTV', 'COROMANDEL', 'MOIL', 'MAZDOCK', 'HOMEFIRST', 'MHRIL', 'SUPRAJIT']
['TEAMLEASE', 'HEG', 'HINDPETRO', 'ZYDUSLIFE', 'EPL', 'NCC']
['COROMANDEL', 'MOIL', 'MAZDOCK', 'HOMEFIRST', 'MHRIL', 'SUPRAJIT']
['TEAMLEASE', 'HEG', 'HINDPETRO', 'ZYDUSLIFE', 'EPL', 'NCC']
sending message
['SUNTV', 'COROMANDEL', 'MOIL', 'MAZDOCK', 'HOMEFIRST', 'MHRIL', 'SUPRAJIT']
['TEAMLEASE', 'HEG', 'HINDPETRO', 'DRREDDY', 'ZYDUSLIFE', 'COFORGE', 'EPL', 'NCC']
['SUNTV', 'MOIL', 'MAZDOCK', 'HOMEFIRST', 'MHRIL', 'SUPRAJIT']
['TEAMLEASE', 'HEG', 'HINDPETRO', 'DRREDDY', 'ZYDUSLIFE', 'EPL', 'NCC']
sending message
['SUNTV', 'MOIL', 'REDINGTON', 'MAZDOCK', 'HOMEFIRST', 'MHRIL', 'SUPRAJIT']
['TEAMLEASE', 'HEG', 'HINDPETRO', 'DRREDDY', 'ZYDUSLIFE', 'COFORGE', 'EPL', 'NCC']
['SUNTV', 'MOIL', 'REDINGTON', 'MAZDOCK', 'HOMEFIRST', 'MHRIL', 'SUPRAJIT']
['TEAMLEASE', 'HEG', 'HINDPETRO', 'DRREDDY', 'ZYDUSLIFE', 'EPL', 'NCC']
sending message
['SUNTV', 'COROMANDEL', 'MOIL', 'MAZDOCK', 'HOMEFIRST', 'MHRIL', 'SUPRAJIT']
['TEAM

KeyboardInterrupt: 

## 4 consecutive candle 

In [12]:
df_all['per_chg'] = abs(df_all['Open'] - df_all['Close'])  / df_all['Close']

In [21]:
def four_candle_setup(df,per_chng = 0.0009):
    # 1. get % chg in open close 
    # 2. consecutive 4 true case 
    df['fst'] = df['per_chg'].shift(-1)
    df['scnd'] = df['per_chg'].shift(-2)
    df['third'] = df['per_chg'].shift(-3)

    # print(df)
    df['cond_flag'] = np.where(
        (df['fst'] <= per_chng) & 
        (df['scnd'] <= per_chng) & 
        (df['third'] <=per_chng)  & 
        (df['per_chg'] <= per_chng),
        True,
        False
        )
    # print(df)
    return df['cond_flag'].sum()

In [22]:
# df_ind = df_all[(df_all.stock == 'JSWENERGY') & (df_all.date_only == '2023-05-04')]
# df_four_ind = df_ind.groupby(['date_only','stock']).apply(four_candle_setup).reset_index().rename(columns={0:"flag"})

In [23]:
df_part_date = df_all[(df_all.date_only == '2023-05-30')]
df_four = df_part_date.groupby(['date_only','stock']).apply(four_candle_setup).reset_index().rename(columns={0:"flag"})

In [24]:
df_four[(df_four.flag == 1) & (df_four.date_only == '2023-05-30')].stock.unique()

array(['AFFLE', 'BEL', 'BLUEDART', 'CAMS', 'CESC', 'CGCL', 'DBL', 'HDFC',
       'HINDZINC', 'IDEA', 'KEI', 'NCC', 'PRESTIGE', 'SHRIRAMFIN',
       'SOLARINDS', 'SUNTECK', 'TATAMTRDVR', 'TEAMLEASE', 'TEJASNET',
       'THYROCARE', 'TITAN', 'TTML', 'VARROC', 'VINATIORGA', 'VTL',
       'WHIRLPOOL'], dtype=object)

In [ ]:
# df_all[(df_all.stock == "JSWENERGY") & (df_all.date_only == '2023-05-05')].head(50)